In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, svm
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy  as np
import math
import random
from deap import creator,tools,base,algorithms

In [ ]:
## 本问题即为以操作变量改变后含硫量限定在
## 55μg/g以内，尽量降低辛烷值损失30%以上为目标的多目标优化问题，
## 其约束为每个操作变量须在其取值范围内变化。

In [ ]:
# 帕累托最优（Pareto Optimality）来衡量解是否最优
# 在帕累托最优的条件下，是没有办法在不让某一参与资源分配的一方利益受损的情况下，令另一方获得更大利益的。

In [3]:
# 加载样本数据
df = pd.read_excel('D:\桌面\数据挖掘-汽油辛烷值损失模型优化\暑期模拟-3-2022-08-18\模拟练习-3 B题 汽油辛烷值建模\降维后特征.xlsx')
df1 = pd.read_excel('D:\桌面\数据挖掘-汽油辛烷值损失模型优化\暑期模拟-3-2022-08-18\模拟练习-3 B题 汽油辛烷值建模\主要变量操作方案的优化.xlsx')

In [67]:
# 拆分成训练集和测试集，测试集大小为原始数据集大小的 2/10  (即训练集数为260)
df_train=df.loc[0:260,:]
df_test=df.loc[261:324,:]
y_train_1=df["RON损失\n（不是变量）"].loc[0:260]
y_test_1=df["RON损失\n（不是变量）"].loc[261:324]
y_train_2=df["硫含量,μg/g.1"].loc[0:260]
y_test_2=df["硫含量,μg/g.1"].loc[261:324]
df_train=df_train.copy()
df_test=df_test.copy()
x_train=df_train.drop(['RON损失\n（不是变量）','硫含量,μg/g.1'], axis=1)
x_test=df_test.drop(['RON损失\n（不是变量）','硫含量,μg/g.1'], axis=1)


In [83]:
# 支持向量机非线性回归SVR模型
def test_SVR_linear(*data):
    X_train,X_test,y_train=data
    regr=svm.SVR(kernel='linear')
    regr.fit(X_train,y_train)
    y_pred = regr.predict(X_test)
    return y_pred

In [84]:
# 分别调用两个SVR模型
y_pred_1=test_SVR_linear(x_train,x_test,y_train_1)
y_pred_2=test_SVR_linear(x_train,x_test,y_train_2)

In [71]:
# 更新索引
y_test_1.reset_index(drop=True,inplace=True)
y_test_2.reset_index(drop=True,inplace=True)
x_test.reset_index(drop=True,inplace=True)

In [ ]:
# NSGA-II 快速非支配排序
# 优化目标：降低辛烷值RON损失值50%，并输出各个操作变量的优化结果
# 问题要求：在保证产品硫含量不大于5μg/g的前提下，降低样本辛烷值RON损失大于30%


In [72]:
### NSGA-II 快速非支配排序 相关函数
#Function to find index of list
#函数查找列表的索引
def index_of(a,list):
    for i in range(0,len(list)):
        if list[i] == a:
            return i
    return -1

#Function to sort by values 函数根据值排序
def sort_by_values(list1, values):
    sorted_list = []
    while(len(sorted_list)!=len(list1)):
        if index_of(min(values),values) in list1:
            sorted_list.append(index_of(min(values),values))
        values[index_of(min(values),values)] = math.inf
    return sorted_list

#Function to carry out NSGA-II's fast non dominated sort
#函数执行NSGA-II的快速非支配排序
"""基于序列和拥挤距离"""
def fast_non_dominated_sort(values1, values2):
    S=[[] for i in range(0,len(values1))]
    front = [[]]
    n=[0 for i in range(0,len(values1))]
    rank = [0 for i in range(0, len(values1))]

    for p in range(0,len(values1)):
        S[p]=[]
        n[p]=0
        for q in range(0, len(values1)):
             #p > q
            if (values1[p] > values1[q] and values2[p] > values2[q]) or (values1[p] >= values1[q] and values2[p] > values2[q]) or (values1[p] > values1[q] and values2[p] >= values2[q]):
                if q not in S[p]:
                    S[p].append(q)
            elif (values1[q] > values1[p] and values2[q] > values2[p]) or (values1[q] >= values1[p] and values2[q] > values2[p]) or (values1[q] > values1[p] and values2[q] >= values2[p]):
                n[p] = n[p] + 1
        if n[p]==0:
            rank[p] = 0
            if p not in front[0]:
                front[0].append(p)

    i = 0
    while(front[i] != []):
        Q=[]
        for p in front[i]:
            for q in S[p]:
                n[q] =n[q] - 1
                if( n[q]==0):
                    rank[q]=i+1
                    if q not in Q:
                        Q.append(q)
        i = i+1
        front.append(Q)

    del front[len(front)-1]

    return front

#Function to calculate crowding distance
#计算拥挤距离的函数
def crowding_distance(values1, values2, front):
    distance = [0 for i in range(0,len(front))]
    sorted1 = sort_by_values(front, values1[:])
    sorted2 = sort_by_values(front, values2[:])
    distance[0] = 4444444444444444
    distance[len(front) - 1] = 4444444444444444
    for k in range(1,len(front)-1):
        distance[k] = distance[k]+ (values1[sorted1[k+1]] - values2[sorted1[k-1]])/(max(values1)-min(values1))
    for k in range(1,len(front)-1):
        distance[k] = distance[k]+ (values1[sorted2[k+1]] - values2[sorted2[k-1]])/(max(values2)-min(values2))
    return distance

#Function to carry out the crossover
#函数进行交叉
def crossover(a,b):
    r=random.random()
    if r>0.5:
        return mutation((a+b)/2)
    else:
        return mutation((a-b)/2)

#Function to carry out the mutation operator
#函数进行变异操作
def mutation(solution):
    mutation_prob = random.random()
    if mutation_prob <1:
        solution = min_x+(max_x-min_x)*random.random()
    return solution


In [73]:
#Main program starts here
pop_size = 20
max_gen = 921

#Initialization
min_x=-55
max_x=55
solution=[min_x+(max_x-min_x)*random.random() for i in range(0,pop_size)]
gen_no=0
while(gen_no<max_gen):
#     function1_values = [function1(solution[i])for i in range(0,pop_size)]
#     function2_values = [function2(solution[i])for i in range(0,pop_size)]
    non_dominated_sorted_solution = fast_non_dominated_sort(y_pred_1[:],y_pred_2[:])
    print("The best front for Generation number ",gen_no, " is")
    for valuez in non_dominated_sorted_solution[0]:
        print(round(solution[valuez],3),end=" ")
    print("\n")
    crowding_distance_values=[]
    for i in range(0,len(non_dominated_sorted_solution)):
        crowding_distance_values.append(crowding_distance(function1_values[:],function2_values[:],non_dominated_sorted_solution[i][:]))
    solution2 = solution[:]

    #Generating offsprings
    while(len(solution2)!=2*pop_size):
        a1 = random.randint(0,pop_size-1)
        b1 = random.randint(0,pop_size-1)
        solution2.append(crossover(solution[a1],solution[b1]))
#     function1_values2 = [function1(solution2[i])for i in range(0,2*pop_size)]
#     function2_values2 = [function2(solution2[i])for i in range(0,2*pop_size)]
    non_dominated_sorted_solution2 = fast_non_dominated_sort(y_pred_1[:],y_pred_2[:])
    crowding_distance_values2=[]
    for i in range(0,len(non_dominated_sorted_solution2)):
        crowding_distance_values2.append(crowding_distance(function1_values2[:],function2_values2[:],non_dominated_sorted_solution2[i][:]))
    new_solution= []
    for i in range(0,len(non_dominated_sorted_solution2)):
        non_dominated_sorted_solution2_1 = [index_of(non_dominated_sorted_solution2[i][j],non_dominated_sorted_solution2[i] ) for j in range(0,len(non_dominated_sorted_solution2[i]))]
        front22 = sort_by_values(non_dominated_sorted_solution2_1[:], crowding_distance_values2[i][:])
        front = [non_dominated_sorted_solution2[i][front22[j]] for j in range(0,len(non_dominated_sorted_solution2[i]))]
        front.reverse()
        for value in front:
            new_solution.append(value)
            if(len(new_solution)==pop_size):
                break
        if (len(new_solution) == pop_size):
            break
    solution = [solution2[i] for i in new_solution]
    gen_no = gen_no + 1

#Lets plot the final front now
# function1 = [i * -1 for i in function1_values]
# function2 = [j * -1 for j in function2_values]
plt.xlabel('Function 1', fontsize=15)
plt.ylabel('Function 2', fontsize=15)
plt.scatter(y_pred_1, y_pred_2)
plt.show()

The best front for Generation number  0  is
-23.167 2.329 

IndexError: list index out of range

In [88]:
# 定义问题
creator.create('MultiObjMin',base.Fitness,weights=(-1.0,-1.0))
creator.create('Individual',list,fitness=creator.MultiObjMin)

# 个体编码
def uniform(low,up):
    # 均匀分布生成个体
    return [random.uniform(a,b) for a,b in zip(low,up)]

pop_size = 100
NDim = 2
# 变量下界
low = [0] * NDim
# 变量上界
up = [400] * NDim
# 生成个体
toolbox = base.Toolbox()
toolbox.register('Attr_float',uniform,low,up)
toolbox.register('Individual',tools.initIterate,creator.Individual,toolbox.Attr_float)
# 生成种群
toolbox.register('Population',tools.initRepeat,list,toolbox.Individual)
pop = toolbox.Population(n = pop_size)

# ZDT3评价函数,ind长度为2  ((((目标函数为最大化问题时，适应度函数等于目标函数；当目标函数为最小化问题时，适应度函数等于-1*目标函数)))
def ZDT3(ind):
    
    a=test_SVR_linear(x_train,ind,y_train_1)
    b=-test_SVR_linear(x_train,x_test,y_train_2)
    f1=a[0]
    f2=b[0]
    return f1,f2

# 注册工具
toolbox.register('evaluate',ZDT3)
# 锦标赛选择
toolbox.register('selectGen1',tools.selTournament,tournsize=2)
# selTournamentDCD(individuals, k)
toolbox.register('select',tools.selTournamentDCD)
# tools.cxSimulatedBinaryBounded(ind1, ind2, eta, low, up)
toolbox.register('mate',tools.cxSimulatedBinaryBounded,eta=20.0,low=low,up=up)
# 变异 - 多项式变异
toolbox.register('mutate',tools.mutPolynomialBounded,eta=20.0,low=low,up=up,indpb=1.0/NDim)

# 用数据记录算法迭代过程
# 创建统计信息对象
stats = tools.Statistics(key= lambda ind : ind.fitness.values)
stats.register('avg',np.mean)
stats.register('std',np.std)
stats.register('min',np.min)
stats.register('max',np.max)

# 创建日志对象
logbook = tools.Logbook()


# 遗传算法主程序
# 参数设置
maxGen = 50
cxProb = 0.7
mutateProb = 0.2

# 遗传算法迭代
# 第一代
fitnesses = map(toolbox.evaluate,pop)
for ind,fit in zip(pop,fitnesses):
    ind.fitness.values = fit
record = stats.compile(pop)
logbook.record(gen=0,**record)

# 快速非支配排序操作
fronts = tools.emo.sortNondominated(pop,k=pop_size)
# 将每个个体的适应度设置为pareto前沿的次序
for idx,front in enumerate(fronts):
    for ind in front:
        ind.fitness.values = (idx + 1),

# 创建子代
offspring = toolbox.selectGen1(pop,pop_size)    # 锦标赛选择
# algorithms.varAnd进化算法的一部分，仅应用变化部分（交叉和变异）,克隆了个体，因此返回的种群独立于输入种群
offspring = algorithms.varAnd(offspring,toolbox,cxProb,mutateProb)

# 第二代之后的迭代
for gen in range(1,maxGen+1):
    # 合并父代与子代
    combinedPop = pop + offspring
    # 评价族群-更新新族群的适应度
    fitnesses = map(toolbox.evaluate,combinedPop)
    for ind,fit in zip(combinedPop,fitnesses):
        ind.fitness.values = fit

    # 快速非支配排序
    fronts = tools.emo.sortNondominated(combinedPop,k=pop_size,first_front_only=False)
    
    # 拥挤距离计算
    for front in fronts:
        tools.emo.assignCrowdingDist(front)

    # 环境选择--精英保留
    pop = []
    for front in fronts:
        pop += front

    # 复制
    pop = toolbox.clone(pop)
    # 基于拥挤度的选择函数用来实现精英保存策略
    pop = tools.selNSGA2(pop,k=pop_size,nd='standard')

    # 创建子代
    offspring = toolbox.select(pop,pop_size)
    offspring = toolbox.clone(offspring)
    offspring = algorithms.varAnd(offspring,toolbox,cxProb,mutateProb)

    # 记录数据-将stats的注册功能应用于pop，并作为字典返回
    record = stats.compile(pop)
    logbook.record(gen = gen,**record)

# 输出计算过程
logbook.header = 'gen','avg','std','min','max'
print(logbook)

# 输出最优解
bestInd = tools.selBest(pop,1)[0]
bestFit = bestInd.fitness.values
print('当前最优解:',bestInd)
print('对应的函数最小值为:',bestFit)

# front = tools.emo.sortNondominated(pop,len(pop))[0]
# gridPop
# for ind in front:
#     plt.plot(ind.fitness.values[0],ind.fitness.values[1],'r.',ms=2)
# plt.xlabel('f_1')
# plt.ylabel('f_2')
# plt.tight_layout()
# plt.show()
plt.plot(y_test)
plt.plot(f1)
fig = plt.gcf()
# fig.set_size_inches(20, 5)
plt.xlabel('Serial Number')
plt.ylabel('Dissolved Oxygen')

AssertionError: Assigned values have not the same length than fitness weights